In [48]:
import os
import pandas as pd
from pandas import read_csv, read_excel, DataFrame
from sae import COTSCar
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum as qsum
import numpy as np
from typing import List

import sys
sys.path.append('../')
import saedfsc

Price of the car.

In [49]:
maxProductPrice = 400000

Information about customers

In [50]:
customers = saedfsc.customers['Name'].to_list()
cQty = dict(zip(saedfsc.customers['Name'], saedfsc.customers['Quantity'])) # customer quantities
cPriceFocus = dict(zip(saedfsc.customers['Name'], saedfsc.customers['PriceFocus']))
name_weights_partworth_dict = saedfsc.customers.set_index('Name')['PartworthUtilityWeights'].to_dict()
name_weights_perf_dict = saedfsc.customers.set_index('Name')['PerformanceUtilityWeights'].to_dict()
cWtsPartworth = {c : np.fromstring(name_weights_partworth_dict[c].strip('[]'), sep=',') for c in name_weights_partworth_dict}
cWtsPerf = {c : np.fromstring(name_weights_perf_dict[c].strip('[]'), sep=',') for c in name_weights_perf_dict}

Methods to help compute the market share for each customer type (based on logit model).

In [51]:
def getTotalUtilityForCustomer(car : COTSCar, c : str, utilFn = 'partworth'):
    total_utility = 0
    pricePerf = (maxProductPrice - car.price) / maxProductPrice # Normalized price (0 is best, 1 is worst)
    if utilFn == 'partworth':
        product_utility = car.partworth_objectives(weights=cWtsPartworth[c])[0]
        total_utility += (1-cPriceFocus[c])*product_utility - cPriceFocus[c]*pricePerf
    elif utilFn == 'performance':
        product_utility = car.objectives(weights=cWtsPerf[c])[0]
        total_utility += (1-cPriceFocus[c])*product_utility + cPriceFocus[c]*pricePerf
    return total_utility

def getMarketShare(car : COTSCar, c : str, competitors : List[COTSCar], utilFn = 'partworth'): # based on logit model of demand
    carUtility = getTotalUtilityForCustomer(car, c, utilFn)
    totalCompetitorUtility = sum([getTotalUtilityForCustomer(competitorCar, c, utilFn) for competitorCar in competitors])
    return carUtility / (totalCompetitorUtility + carUtility)

Create a set of competitors.

In [52]:
profitMargin = 0.1
competitors = []
for i in range(3):
    competitorCar = COTSCar()
    competitorCar.price = (1+profitMargin)*competitorCar.cost()
    competitorCar.name = "Competitor Car " + str(i)
    competitors.append(competitorCar)

names = [car.name for car in competitors]
designs = [car.vector for car in competitors]
salesPrices = ['$' + format(car.price, ",.2f") for car in competitors]
costs = ['$' + format(car.cost(), ",.2f") for car in competitors]
competitorsDF = pd.DataFrame({
    'Name': names,
    'Design': designs,
    'SalesPrice': salesPrices,
    'ProductCost': costs
})
competitorsDF

,Name,Design,SalesPrice,ProductCost
0,Competitor Car 0,"[5, 7, 0, 7, 11, 1, 5, 1, 7, 2, 2, 37, 0, 2, 1...","$4,914.85","$4,468.05"
1,Competitor Car 1,"[4, 10, 9, 8, 12, 4, 3, 12, 30, 2, 48, 153, 0,...","$110,221.47","$100,201.34"
2,Competitor Car 2,"[3, 11, 4, 5, 5, 3, 3, 16, 26, 1, 13, 51, 1, 1...","$24,974.52","$22,704.11"


## Partworth Utility

Create a random car and compute market share

In [53]:
carDesignVec = [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
product = COTSCar()
product.vector = carDesignVec
product.price = (1 + profitMargin)*product.cost()
print("OUR CAR:") 
print("Design: ", product.vector)
print("Sales price: $", format(product.price, ",.2f"))
print("Cost: $", format(product.cost(), ",.2f"))

qtys = [cQty[c] for c in customers]
utilities = [getTotalUtilityForCustomer(product, c) for c in customers]
market_shares = [getMarketShare(product, c, competitors) for c in customers]
expected_customer_demand = [getMarketShare(product, c, competitors)*cQty[c] for c in customers]

df = pd.DataFrame({
    'Name': customers,
    'Quantity': qtys,
    'Utility for our car': utilities,
    'Market share': market_shares,
    'Expected demand': expected_customer_demand
})
total_profit = product.price*df['Expected demand'].sum()
print("Total expected profit: $", format(total_profit, ",.2f"))
print("Market information for our car:")
df

OUR CAR:
Design:  [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
Sales price: $ 9,323.97
Cost: $ 8,476.34
Total expected profit: $ 44,691,069.92
Market information for our car:


,Name,Quantity,Utility for our car,Market share,Expected demand
0,CustomerType1,469,-1.966241,0.476676,223.561197
1,CustomerType2,929,-1.014508,0.275714,256.138526
2,CustomerType3,294,-1.765798,0.381552,112.176288
3,CustomerType4,834,-1.599019,0.422365,352.252792
4,CustomerType5,733,-1.947100,0.427901,313.651345
5,CustomerType6,35,-1.586768,0.321175,11.241117
6,CustomerType7,239,-1.798625,0.289480,69.185812
7,CustomerType8,564,-1.765243,0.376613,212.409720
8,CustomerType9,927,-1.927681,0.395932,367.028562
9,CustomerType10,366,-2.158297,0.495084,181.200671


## Performance Utility

In [58]:
carDesignVec = [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
product = COTSCar()
print("COTSCar vec:", product.get_vec())
print("Car vec:", product.car.get_vec())
print("Cost: $", format(product.cost(), ",.2f"))
product.set_vec(carDesignVec)
print("COTSCar vec:", product.get_vec())
print("Car vec:", product.car.get_vec())
print("Cost: $", format(product.cost(), ",.2f"))
product.price = (1 + profitMargin)*product.cost()
print("OUR CAR:") 
print("Design: ", product.vector)
print("Sales price: $", format(product.price, ",.2f"))
print("Cost: $", format(product.cost(), ",.2f"))

qtys = [cQty[c] for c in customers]
utilities = [getTotalUtilityForCustomer(product, c, 'performance') for c in customers]
market_shares = [getMarketShare(product, c, competitors, 'performance') for c in customers]
expected_customer_demand = [getMarketShare(product, c, competitors, 'performance')*cQty[c] for c in customers]

df = pd.DataFrame({
    'Name': customers,
    'Quantity': qtys,
    'Utility for our car': utilities,
    'Market share': market_shares,
    'Expected demand': expected_customer_demand
})
total_profit = product.price*df['Expected demand'].sum()
print("Total expected profit: $", format(total_profit, ",.2f"))
print("Market information for our car:")
df

COTSCar vec: [0, 0, 12, 9, 5, 4, 5, 9, 18, 0, 63, 70, 3, 2, 94, 46, 36]
Car vec: [0.3625, 0.15, 0.3926990815, 0.1125, 0.275, 1.65, 0.3926990815, 0.1125, 0.30000000000000004, 0.225, 0.3926990815, 0.894, 0.894, 0.6120817771521734, 1.8421606282386331, 1.3937518499228556, 0.007414368836192332, 0.3538183511215326, 0.3850508984033272, 5, 6, 7, 2, 7, 0, 3, 13, 20, 3, 0.5484287498156001, 0.7679158678739382, 0.17545563948117573, 0.10611775895215035, 0.2137548982186032, 0.7295061566429835, 0.24323006416003673, 0.9846194623778417, 0.2967268421009293, 0.4328799518163817]
Cost: $ 24,462.04
COTSCar vec: [0, 0, 12, 9, 5, 4, 5, 9, 18, 0, 63, 70, 3, 2, 94, 46, 36]
Car vec: [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4, 0.3538183511215326, 0.3850508984033272, 5, 6, 7, 2, 7, 0, 3, 13, 20, 3, 0.5484287498156001, 0.7679158678739382, 0.17545563948117573, 0.10611775895215035, 0.2137548982186032, 0.7295061566429835, 0.24323006416003673, 0.9846194623778417, 0.2967268421009293, 0.4328799518163817]
C

,Name,Quantity,Utility for our car,Market share,Expected demand
0,CustomerType1,469,-1109.413198,1.005331,471.500110
1,CustomerType2,929,-741.089241,1.004015,932.729603
2,CustomerType3,294,-235.965010,1.020732,300.095214
3,CustomerType4,834,-223.733729,1.022235,852.543631
4,CustomerType5,733,-565.066516,1.007821,738.732547
5,CustomerType6,35,-694.445782,1.009928,35.347491
6,CustomerType7,239,-646.083250,1.006490,240.551060
7,CustomerType8,564,-825.828421,1.004220,566.379971
8,CustomerType9,927,-616.806829,1.008161,934.565574
9,CustomerType10,366,-48.608815,1.132121,414.356295
